## Reading of ASCII files created for cams diagnostics tool

In [19]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from helper_funcs import read_file_custom, read_var_info_michaels_excel
VERBOSE = True

### Paths and file definitions

Please change accordingly if you execute this notebook on your local machine.

In [ ]:
data_dir = "./data/michael_ascii_read/"
case_ok = "./data/from_ada/table_GLBL_ANN_obs_FIXED.asc"

michaels_excel = data_dir + "obs-comp-noresmversions.xlsx"

### Importing supplementary information from Excel table

Let's begin with reading the variable information from the excel table. Note that this is not strictly required but helps us below to display the results in a more intuitive manner, when analysing the data. The custom method that we use is:

In [2]:
help(read_var_info_michaels_excel)

Help on function read_var_info_michaels_excel in module helper_funcs:

read_var_info_michaels_excel(xlspath)
    Read short description strings for variables
    
    The strings are available for some of the variables in Michaels analysis
    excel table (column 3, sheet "DATA")
    
    Parameters
    ----------
    xlspath : location of excel spreadsheet
    
    Returns
    -------
    dict 
        dictionary containing all variable names (keys) and corresponding
        description strings (if applicable, else empty string)



Load the information.

In [3]:
var_info_dict = read_var_info_michaels_excel(michaels_excel)
var_info_dict

{'AODDUST': '',
 'AODVIS': '',
 'CLDTOT_CLOUDSAT': '',
 'CLDTOT_ISCCP': 'Total cloud cover',
 'FLDS_ISCCP': 'LW down SRF',
 'FLNS_ISCCP': 'LW net SRF',
 'FLNT_CAM': '',
 'FLUTC_CERES': '',
 'FLUTC_CERES-EBAF': 'LW up Top Clearsky',
 'FLUTC_ERBE': '',
 'FLUT_CERES': '',
 'FLUT_CERES-EBAF': 'LW up Top',
 'FLUT_ERBE': '',
 'FSDS_ISCCP': 'SW down SRF',
 'FSNS_ISCCP': 'SW net SRF',
 'FSNS_LARYEA': '',
 'FSNTOAC_CERES': 'SW net TOA clearsky',
 'FSNTOAC_ERBE': '',
 'FSNTOA_CERES': 'SW net TOA',
 'FSNTOA_ERBE': '',
 'FSNT_CAM': '',
 'LHFLX_ERA40': '',
 'LHFLX_JRA25': 'Lat Heat Flux',
 'LHFLX_WHOI': '',
 'LWCF_CERES': '',
 'LWCF_CERES-EBAF': 'LW Cloud Forc',
 'LWCF_ERBE': '',
 'PRECT_GPCP': 'Precipitation',
 'PREH2O_AIRS': '',
 'PREH2O_ERA40': 'Precipitable water',
 'PREH2O_ERAI': '',
 'PREH2O_JRA25': '',
 'PREH2O_NVAP': '',
 'PSL_ERAI': '',
 'PSL_JRA25': 'SeaLev pressure',
 'RESSURF': 'SRF net flux',
 'RESTOA_CERES-EBAF': 'TOA  net flux',
 'RESTOA_ERBE': '',
 'RESTOM': 'TOmodel net flux',
 'SH

### Search and load ASCII files (either using .asc or .webarchive file type)

In [4]:
files = glob(data_dir + "*.webarchive")

for file in files:
    print(file)
    
test_file = files[0]

print("TEST FILE: {}".format(os.path.basename(test_file)))

./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 80-110).webarchive
./data/michael_ascii_read/N1850_f09_tn14_230218 (yrs 1-20).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 185-215).webarchive
./data/michael_ascii_read/N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r265_ctrl_20180411 (yrs 90-120).webarchive
TEST FILE: N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive


Try read first file as is with pandas

In [5]:
try:
    frame = pd.read_csv(test_file, encoding="latin-1")
except Exception as e:
    print(repr(e))
frame.head()

,bplist00Ñ_WebMainResourceÕ
0,_WebResourceTextEncodingName^WebResourceUR...
1,TEST CASE: N1850_f19_tn14_r227_ctrl (yrs 310-340)
2,CONTROL CASE: OBS data
3,Variable N1850_f19_tn14_r227_ctrl OBS...
4,...


This did not work (it basically did not separate the individual columns). The same is the case for the file that includes a whitespace at the problematic variables.

In [6]:
frame = pd.read_csv(case_ok)
frame.head()
frame.shape

(67, 1)

This did not really work since the data is not splitted by columns but includes one column containing the content of each row. The reading has to be done from scratch, especially also because there is some variables with too long names (e.g. L.25 and L. 28) that stick together the first two columns. 

This folder contains a file ``helper_funcs.py`` in which I defined a custom read function ``read_file_custom`` that can convert these files into pandas dataframes.

In [7]:
help(read_file_custom)

Help on function read_file_custom in module helper_funcs:

read_file_custom(fpath, var_info_dict=None, verbose=False)
    Custom ASCII conversion method 
    
    Parameters
    ----------
    fpath : str
        path to file location
    var_info_dict : dict
        optinal dictionary that contains description strings for each of the 
        variables (e.g. retrieved using :func:`read_var_info_michaels_excel`)
    verbose : bool
        if True, print output (defaults to False)
    Returns
    -------
    Dataframe 
        pandas data frame ready for further analysis



Now load the first file using this function (without providing the optional parameter *var_info_dict*, i.e. the info from Michael's Excel sheet.

In [8]:
df = read_file_custom(test_file, VERBOSE)
df

Flag  \
Run                       Years   Variable           Description          
N1850_f19_tn14_r227_ctrl  310-340 RESTOM                          False   
                                  RESSURF                         False   
                                  RESTOA_CERES-EBAF               False   
                                  RESTOA_ERBE                     False   
                                  SOLIN_CERES-EBAF                False   
                                  SOLIN_CERES                     False   
                                  CLDTOT_ISCCP                    False   
                                  CLDTOT_CLOUDSAT                 False   
                                  FLDS_ISCCP                      False   
                                  FLNS_ISCCP                      False   
                                  FLUT_CERES-EBAF                 False   
                                  FLUT_CERES                      False   
                                  FLUT_ERBE                       False   
                                  FLUTC_CERES-EBAF                False   
                                  FLUTC_CERES                     False   
                                  FLUTC_ERBE                      False   
                                  FLNT_CAM                        False   
                                  FSDS_ISCCP                      False   
                                  FSNS_ISCCP                      False   
                                  FSNS_LARYEA                     False   
                                  FSNTOA_CERES-EBAF               False   
                                  FSNTOA_CERES                    False   
                                  FSNTOA_ERBE                     False   
                                  FSNTOAC_CERES-EBAF              False   
                                  FSNTOAC_CERES                   False   
                                  FSNTOAC_ERBE                    False   
                                  FSNT_CAM                        False   
                                  LHFLX_JRA25                     False   
                                  LHFLX_ERA40                     False   
                                  LHFLX_WHOI                      False   
...                                                                 ...   
                                  PRECT_GPCP                      False   
                                  PREH2O_NVAP                     False   
                                  PREH2O_AIRS                     False   
                                  PREH2O_JRA25                    False   
                                  PREH2O_ERAI                     False   
                                  PREH2O_ERA40                    False   
                                  PSL_JRA25                       False   
                                  PSL_ERAI                        False   
                                  SHFLX_JRA25                     False   
                                  SHFLX_NCEP                      False   
                                  SHFLX_LARYEA                    False   
                                  STRESS_MAG_ERS                  False   
                                  STRESS_MAG_LARYEA               False   
                                  STRESS_MAG_JRA25                False   
                                  SWCF_CERES-EBAF                 False   
                                  SWCF_CERES                      False   
                                  SWCF_ERBE                       False   
                                  AODVIS                          False   
                                  AODDUST                         False   
                                  SST_HADISST                     False   
                                  SST_HADISST_PI                  False   
                 

That worked, you can see that the column *Description* is empty and that all flags are set `False`. Here in the default reading function the flag is set True, if the ``var_info_dict`` is provided and information for a given variable is available. We illustrate that in the following, our second example, where we load the corrected ascii file and provide the info dictionary that we imported from the Excel sheet.

In [9]:
df1 = read_file_custom(case_ok, var_info_dict, VERBOSE)
df1

Ignoring line: DIAG SET 1: ANN MEANS GLOBAL
Ignoring line:  
Ignoring line:  
Ignoring line:  
Problem case FSNTOA_CERES-EBAF
Problem case FSNTOAC_CERES-EBAF
Test case: N1850_f19_tn14_r265_ctrl_20180411 
Control case: OBS data


Flag  \
Run                                Years   Variable           Description                  
N1850_f19_tn14_r265_ctrl_20180411  150-180 RESTOM             TOmodel net flux      True   
                                           RESSURF            SRF net flux          True   
                                           RESTOA_CERES-EBAF  TOA  net flux         True   
                                           RESTOA_ERBE                             False   
                                           SOLIN_CERES-EBAF                        False   
                                           SOLIN_CERES                             False   
                                           CLDTOT_ISCCP       Total cloud cover     True   
                                           CLDTOT_CLOUDSAT                         False   
                                           FLDS_ISCCP         LW down SRF           True   
                                           FLNS_ISCCP         LW net SRF            True   
                                           FLUT_CERES-EBAF    LW up Top             True   
                                           FLUT_CERES                              False   
                                           FLUT_ERBE                               False   
                                           FLUTC_CERES-EBAF   LW up Top Clearsky    True   
                                           FLUTC_CERES                             False   
                                           FLUTC_ERBE                              False   
                                           FLNT_CAM                                False   
                                           FSDS_ISCCP         SW down SRF           True   
                                           FSNS_ISCCP         SW net SRF            True   
                                           FSNS_LARYEA                             False   
                                           FSNTOA_CERES-EBAF                       False   
                                           FSNTOA_CERES       SW net TOA            True   
                                           FSNTOA_ERBE                             False   
                                           FSNTOAC_CERES-EBAF                      False   
                                           FSNTOAC_CERES      SW net TOA clearsky   True   
                                           FSNTOAC_ERBE                            False   
                                           FSNT_CAM                                False   
                                           LHFLX_JRA25        Lat Heat Flux         True   
                                           LHFLX_ERA40                             False   
                                           LHFLX_WHOI                              False   
...                                                                                  ...   
                                           PRECT_GPCP         Precipitation         True   
                                           PREH2O_NVAP                             False   
                                           PREH2O_AIRS                             False   
                                           PREH2O_JRA25                            False   
                                           PREH2O_ERAI                             False   
                                           PREH2O_ERA40       Precipitable water    True   
                                           PSL_JRA25          SeaLev pressure       True   
                                           PSL_ERAI                                False   
                                           SHFLX_JRA25        Sens Heat Flux        True   
                                           SHFLX_NCEP                              False   
                                           SHFLX_LARYEA                            False   
                                    

Reading worked, check, if both dataframes have the same dimension.

In [10]:
print(df.shape)
print(df1.shape)

(63, 5)
(63, 5)


### Importing multiple result files and concatenating them into one Dataframe

Now we have a method that can import the results from a single run into a datframe that can be used for further analysis. In the following, we basically do this for all available files and put the results into one big `Dataframe`.

In [11]:
dfs = []
for file in files:
    df = read_file_custom(file, var_info_dict, verbose=False)
    #print(df.head(), "\n\n")
    dfs.append(df)
    if VERBOSE:
        print("Number of variables / values")
        print(df.shape)        

Number of variables / values
(63, 5)
Number of variables / values
(63, 5)
Number of variables / values
(63, 5)
Number of variables / values
(63, 5)
Number of variables / values
(63, 5)
Number of variables / values
(63, 5)


In [12]:
merged = pd.concat(dfs)
merged

Flag  \
Run                                Years   Variable           Description                  
N1850_f19_tn14_r227_ctrl           310-340 RESTOM             TOmodel net flux      True   
                                           RESSURF            SRF net flux          True   
                                           RESTOA_CERES-EBAF  TOA  net flux         True   
                                           RESTOA_ERBE                             False   
                                           SOLIN_CERES-EBAF                        False   
                                           SOLIN_CERES                             False   
                                           CLDTOT_ISCCP       Total cloud cover     True   
                                           CLDTOT_CLOUDSAT                         False   
                                           FLDS_ISCCP         LW down SRF           True   
                                           FLNS_ISCCP         LW net SRF            True   
                                           FLUT_CERES-EBAF    LW up Top             True   
                                           FLUT_CERES                              False   
                                           FLUT_ERBE                               False   
                                           FLUTC_CERES-EBAF   LW up Top Clearsky    True   
                                           FLUTC_CERES                             False   
                                           FLUTC_ERBE                              False   
                                           FLNT_CAM                                False   
                                           FSDS_ISCCP         SW down SRF           True   
                                           FSNS_ISCCP         SW net SRF            True   
                                           FSNS_LARYEA                             False   
                                           FSNTOA_CERES-EBAF                       False   
                                           FSNTOA_CERES       SW net TOA            True   
                                           FSNTOA_ERBE                             False   
                                           FSNTOAC_CERES-EBAF                      False   
                                           FSNTOAC_CERES      SW net TOA clearsky   True   
                                           FSNTOAC_ERBE                            False   
                                           FSNT_CAM                                False   
                                           LHFLX_JRA25        Lat Heat Flux         True   
                                           LHFLX_ERA40                             False   
                                           LHFLX_WHOI                              False   
...                                                                                  ...   
N1850_f19_tn14_r265_ctrl_20180411  90-120  PRECT_GPCP         Precipitation         True   
                                           PREH2O_NVAP                             False   
                                           PREH2O_AIRS                             False   
                                           PREH2O_JRA25                            False   
                                           PREH2O_ERAI                             False   
                                           PREH2O_ERA40       Precipitable water    True   
                                           PSL_JRA25          SeaLev pressure       True   
                                           PSL_ERAI                                False   
                                           SHFLX_JRA25        Sens Heat Flux        True   
                                           SHFLX_NCEP                              False   
                                           SHFLX_LARYEA                            False   
                                    

### Working with the imported data

Now we can use the `Flag` column to select only the variables that we are interested in.


In [13]:
flagged = merged[merged["Flag"]]
flagged

Flag  \
Run                                Years   Variable          Description                 
N1850_f19_tn14_r227_ctrl           310-340 RESTOM            TOmodel net flux     True   
                                           RESSURF           SRF net flux         True   
                                           RESTOA_CERES-EBAF TOA  net flux        True   
                                           CLDTOT_ISCCP      Total cloud cover    True   
                                           FLDS_ISCCP        LW down SRF          True   
                                           FLNS_ISCCP        LW net SRF           True   
                                           FLUT_CERES-EBAF   LW up Top            True   
                                           FLUTC_CERES-EBAF  LW up Top Clearsky   True   
                                           FSDS_ISCCP        SW down SRF          True   
                                           FSNS_ISCCP        SW net SRF           True   
                                           FSNTOA_CERES      SW net TOA           True   
                                           FSNTOAC_CERES     SW net TOA clearsky  True   
                                           LHFLX_JRA25       Lat Heat Flux        True   
                                           LWCF_CERES-EBAF   LW Cloud Forc        True   
                                           PRECT_GPCP        Precipitation        True   
                                           PREH2O_ERA40      Precipitable water   True   
                                           PSL_JRA25         SeaLev pressure      True   
                                           SHFLX_JRA25       Sens Heat Flux       True   
                                           SWCF_CERES-EBAF   SW Cloud Forc        True   
                                           SST_HADISST_PI    SST preindustrial    True   
                                           TREFHT_JRA25      2m Temp              True   
                                           TS_NCEP           2m Temp              True   
                                           TS_LAND_NCEP      2m Temp land         True   
                                           U_200_JRA25       Zonal Wind JRA25     True   
                                           Z3_500_JRA25      Geop Height  JRA25   True   
                                   80-110  RESTOM            TOmodel net flux     True   
                                           RESSURF           SRF net flux         True   
                                           RESTOA_CERES-EBAF TOA  net flux        True   
                                           CLDTOT_ISCCP      Total cloud cover    True   
                                           FLDS_ISCCP        LW down SRF          True   
...                                                                                ...   
N1850C53CLM45L32_f09_tn11_191017   71-100  TREFHT_JRA25      2m Temp              True   
                                           TS_NCEP           2m Temp              True   
                                           TS_LAND_NCEP      2m Temp land         True   
                                           U_200_JRA25       Zonal Wind JRA25     True   
                                           Z3_500_JRA25      Geop Height  JRA25   True   
N1850_f19_tn14_r265_ctrl_20180411  90-120  RESTOM            TOmodel net flux     True   
                                           RESSURF           SRF net flux         True   
                                           RESTOA_CERES-EBAF TOA  net flux        True   
                                           CLDTOT_ISCCP      Total cloud cover    True   
                                           FLDS_ISCCP        LW down SRF          True   
                                           FLNS_ISCCP        LW net SRF           True   
                                           FLUT_CERES-EBAF   LW up Top            True   
                                

For visualisation this display requires a lot of scrolling. We can make the table `broader` by unstacking certain indices, e.g. the two outermost indices `Run` and `Years`.

In [14]:
flagged_unstacked = flagged.unstack(level=(0,1))
flagged_unstacked

Flag         \
Run                                   N1850_f19_tn14_r227_ctrl           
Years                                                   310-340 80-110   
Variable          Description                                            
CLDTOT_ISCCP      Total cloud cover                        True   True   
FLDS_ISCCP        LW down SRF                              True   True   
FLNS_ISCCP        LW net SRF                               True   True   
FLUTC_CERES-EBAF  LW up Top Clearsky                       True   True   
FLUT_CERES-EBAF   LW up Top                                True   True   
FSDS_ISCCP        SW down SRF                              True   True   
FSNS_ISCCP        SW net SRF                               True   True   
FSNTOAC_CERES     SW net TOA clearsky                      True   True   
FSNTOA_CERES      SW net TOA                               True   True   
LHFLX_JRA25       Lat Heat Flux                            True   True   
LWCF_CERES-EBAF   LW Cloud Forc                            True   True   
PRECT_GPCP        Precipitation                            True   True   
PREH2O_ERA40      Precipitable water                       True   True   
PSL_JRA25         SeaLev pressure                          True   True   
RESSURF           SRF net flux                             True   True   
RESTOA_CERES-EBAF TOA  net flux                            True   True   
RESTOM            TOmodel net flux                         True   True   
SHFLX_JRA25       Sens Heat Flux                           True   True   
SST_HADISST_PI    SST preindustrial                        True   True   
SWCF_CERES-EBAF   SW Cloud Forc                            True   True   
TREFHT_JRA25      2m Temp                                  True   True   
TS_LAND_NCEP      2m Temp land                             True   True   
TS_NCEP           2m Temp                                  True   True   
U_200_JRA25       Zonal Wind JRA25                         True   True   
Z3_500_JRA25      Geop Height  JRA25                       True   True   

                                                              \
Run                                   N1850_f09_tn14_230218    
Years                                                   1-20   
Variable          Description                                  
CLDTOT_ISCCP      Total cloud cover                     True   
FLDS_ISCCP        LW down SRF                           True   
FLNS_ISCCP        LW net SRF                            True   
FLUTC_CERES-EBAF  LW up Top Clearsky                    True   
FLUT_CERES-EBAF   LW up Top                             True   
FSDS_ISCCP        SW down SRF                           True   
FSNS_ISCCP        SW net SRF                            True   
FSNTOAC_CERES     SW net TOA clearsky                   True   
FSNTOA_CERES      SW net TOA                            True   
LHFLX_JRA25       Lat Heat Flux                         True   
LWCF_CERES-EBAF   LW Cloud Forc                         True   
PRECT_GPCP        Precipitation                         True   
PREH2O_ERA40      Precipitable water                    True   
PSL_JRA25         SeaLev pressure                       True   
RESSURF           SRF net flux                          True   
RESTOA_CERES-EBAF TOA  net flux                         True   
RESTOM            TOmodel net flux                      True   
SHFLX_JRA25       Sens Heat Flux                        True   
SST_HADISST_PI    SST preindustrial                     True   
SWCF_CERES-EBAF   SW Cloud Forc                         True   
TREFHT_JRA25      2m Temp                               True   
TS_LAND_NCEP      2m Temp land                          True   
TS_NCEP           2m Temp                               True   
U_200_JRA25       Zonal Wind JRA25                      True   
Z3_500_JRA25      Geop Height  JRA25                    True   

                                                       

Well, this is better but also not extremely illustrative / intuitive. It becomes more intuitive if we just look at one parameter that we are interested in (e.g. RMSE). 

#### Extracting the Bias of each model run relative to the observations

Retrieving a table that illustrates the Bias of each run for each flagged variable is straight forward. We just extract the `Bias` column from our flagged frame:

In [15]:
bias_table = flagged_unstacked["Bias"]
bias_table

Run                                   N1850_f19_tn14_r227_ctrl          \
Years                                                   310-340 80-110   
Variable          Description                                            
CLDTOT_ISCCP      Total cloud cover                       1.435  2.157   
FLDS_ISCCP        LW down SRF                            11.499  5.162   
FLNS_ISCCP        LW net SRF                              6.847  7.167   
FLUTC_CERES-EBAF  LW up Top Clearsky                      1.426 -0.786   
FLUT_CERES-EBAF   LW up Top                               2.928  0.660   
FSDS_ISCCP        SW down SRF                             1.228  1.216   
FSNS_ISCCP        SW net SRF                              1.365 -0.017   
FSNTOAC_CERES     SW net TOA clearsky                    -4.183 -5.747   
FSNTOA_CERES      SW net TOA                              0.223 -1.724   
LHFLX_JRA25       Lat Heat Flux                           0.434 -1.695   
LWCF_CERES-EBAF   LW Cloud Forc                          -1.501 -1.446   
PRECT_GPCP        Precipitation                           0.352  0.276   
PREH2O_ERA40      Precipitable water                      3.437  1.922   
PSL_JRA25         SeaLev pressure                         0.513  0.448   
RESSURF           SRF net flux                              NaN    NaN   
RESTOA_CERES-EBAF TOA  net flux                           1.420  1.741   
RESTOM            TOmodel net flux                          NaN    NaN   
SHFLX_JRA25       Sens Heat Flux                          2.907  3.028   
SST_HADISST_PI    SST preindustrial                       2.385  1.756   
SWCF_CERES-EBAF   SW Cloud Forc                           1.466  1.083   
TREFHT_JRA25      2m Temp                                 1.615  0.431   
TS_LAND_NCEP      2m Temp land                            3.945  2.181   
TS_NCEP           2m Temp                                 2.767  1.564   
U_200_JRA25       Zonal Wind JRA25                        3.218  3.340   
Z3_500_JRA25      Geop Height  JRA25                      0.317  0.113   

Run                                   N1850_f09_tn14_230218   \
Years                                                   1-20   
Variable          Description                                  
CLDTOT_ISCCP      Total cloud cover                    1.786   
FLDS_ISCCP        LW down SRF                         -1.799   
FLNS_ISCCP        LW net SRF                           9.699   
FLUTC_CERES-EBAF  LW up Top Clearsky                  -0.986   
FLUT_CERES-EBAF   LW up Top                            1.704   
FSDS_ISCCP        SW down SRF                          3.200   
FSNS_ISCCP        SW net SRF                           2.630   
FSNTOAC_CERES     SW net TOA clearsky                 -5.051   
FSNTOA_CERES      SW net TOA                          -0.338   
LHFLX_JRA25       Lat Heat Flux                       -2.503   
LWCF_CERES-EBAF   LW Cloud Forc                       -2.689   
PRECT_GPCP        Precipitation                        0.246   
PREH2O_ERA40      Precipitable water                  -0.511   
PSL_JRA25         SeaLev pressure                      0.004   
RESSURF           SRF net flux                           NaN   
RESTOA_CERES-EBAF TOA  net flux                        2.083   
RESTOM            TOmodel net flux                       NaN   
SHFLX_JRA25       Sens Heat Flux                       3.621   
SST_HADISST_PI    SST preindustrial                    0.782   
SWCF_CERES-EBAF   SW Cloud Forc                        1.774   
TREFHT_JRA25      2m Temp                             -0.263   
TS_LAND_NCEP      2m Temp land                         1.237   
TS_NCEP           2m Temp                              0.876   
U_200_JRA25       Zonal Wind JRA25                     1.825   
Z3_500_JRA25      Geop Height  JRA25                  -0.090   

Run                                   N1850_f19_tn14_r227_ctrl   \
Years                                                   185-215   


#### Computing RMSE relative error

In the following we extract the subset containing the *RSME* information of the flagged variables for all runs in order to compute the relative error for each run based on the average *RMSE* of all runs:

$$\frac{RMSE_{Run}\,-\,\overline{RMSE_{All\,Runs}}}{\overline{RMSE_{All\,Runs}}}$$


In [16]:
rmse = flagged_unstacked["RMSE"]
rmse

Run                                   N1850_f19_tn14_r227_ctrl           \
Years                                                   310-340  80-110   
Variable          Description                                             
CLDTOT_ISCCP      Total cloud cover                      13.078  12.869   
FLDS_ISCCP        LW down SRF                            17.664  16.720   
FLNS_ISCCP        LW net SRF                             14.174  14.516   
FLUTC_CERES-EBAF  LW up Top Clearsky                      4.738   5.670   
FLUT_CERES-EBAF   LW up Top                               7.467   7.499   
FSDS_ISCCP        SW down SRF                            16.082  16.048   
FSNS_ISCCP        SW net SRF                             13.727  13.705   
FSNTOAC_CERES     SW net TOA clearsky                    15.710  18.506   
FSNTOA_CERES      SW net TOA                             12.125  12.314   
LHFLX_JRA25       Lat Heat Flux                          15.116  15.587   
LWCF_CERES-EBAF   LW Cloud Forc                           5.980   6.018   
PRECT_GPCP        Precipitation                           1.069   1.055   
PREH2O_ERA40      Precipitable water                      4.430   3.351   
PSL_JRA25         SeaLev pressure                         1.939   1.940   
RESSURF           SRF net flux                              NaN     NaN   
RESTOA_CERES-EBAF TOA  net flux                           8.937   8.872   
RESTOM            TOmodel net flux                          NaN     NaN   
SHFLX_JRA25       Sens Heat Flux                         11.138  11.551   
SST_HADISST_PI    SST preindustrial                       2.606   2.135   
SWCF_CERES-EBAF   SW Cloud Forc                          10.563  10.393   
TREFHT_JRA25      2m Temp                                 2.634   2.665   
TS_LAND_NCEP      2m Temp land                            5.229   4.164   
TS_NCEP           2m Temp                                 3.529   3.004   
U_200_JRA25       Zonal Wind JRA25                        4.348   4.335   
Z3_500_JRA25      Geop Height  JRA25                      0.423   0.335   

Run                                   N1850_f09_tn14_230218   \
Years                                                   1-20   
Variable          Description                                  
CLDTOT_ISCCP      Total cloud cover                   11.881   
FLDS_ISCCP        LW down SRF                         15.351   
FLNS_ISCCP        LW net SRF                          14.953   
FLUTC_CERES-EBAF  LW up Top Clearsky                   4.778   
FLUT_CERES-EBAF   LW up Top                            6.169   
FSDS_ISCCP        SW down SRF                         15.089   
FSNS_ISCCP        SW net SRF                          12.632   
FSNTOAC_CERES     SW net TOA clearsky                 17.432   
FSNTOA_CERES      SW net TOA                          10.795   
LHFLX_JRA25       Lat Heat Flux                       14.578   
LWCF_CERES-EBAF   LW Cloud Forc                        5.683   
PRECT_GPCP        Precipitation                        0.891   
PREH2O_ERA40      Precipitable water                   1.920   
PSL_JRA25         SeaLev pressure                      1.512   
RESSURF           SRF net flux                           NaN   
RESTOA_CERES-EBAF TOA  net flux                        8.358   
RESTOM            TOmodel net flux                       NaN   
SHFLX_JRA25       Sens Heat Flux                      11.926   
SST_HADISST_PI    SST preindustrial                    1.223   
SWCF_CERES-EBAF   SW Cloud Forc                        8.854   
TREFHT_JRA25      2m Temp                              1.955   
TS_LAND_NCEP      2m Temp land                         3.132   
TS_NCEP           2m Temp                              2.320   
U_200_JRA25       Zonal Wind JRA25                     2.852   
Z3_500_JRA25      Geop Height  JRA25                   0.192   

Run                                   N1850_f19_tn14_r227_ctrl   \
Years                                  

In [17]:
rmse_mean = rmse.mean(axis=1, skipna=True)
rmse_mean

Variable           Description        
CLDTOT_ISCCP       Total cloud cover      12.438000
FLDS_ISCCP         LW down SRF            16.059000
FLNS_ISCCP         LW net SRF             13.949333
FLUTC_CERES-EBAF   LW up Top Clearsky      5.111333
FLUT_CERES-EBAF    LW up Top               6.962667
FSDS_ISCCP         SW down SRF            15.322500
FSNS_ISCCP         SW net SRF             13.238333
FSNTOAC_CERES      SW net TOA clearsky    17.282000
FSNTOA_CERES       SW net TOA             12.058000
LHFLX_JRA25        Lat Heat Flux          15.426167
LWCF_CERES-EBAF    LW Cloud Forc           5.895000
PRECT_GPCP         Precipitation           1.032500
PREH2O_ERA40       Precipitable water      3.062333
PSL_JRA25          SeaLev pressure         1.707333
RESSURF            SRF net flux                 NaN
RESTOA_CERES-EBAF  TOA  net flux           8.974500
RESTOM             TOmodel net flux             NaN
SHFLX_JRA25        Sens Heat Flux         11.438167
SST_HADISST_PI     SST pr

The next step is (semi) straight forward (we have to use the `div` and `subtract` methods of the Dataframe rather than `/` and `-` operators in order to specify that we want to apply them in the horizontal and not in the vertical direction.

In [18]:
typical_rmse = rmse.subtract(rmse_mean, axis=0).div(rmse_mean, axis=0)
typical_rmse

Run                                   N1850_f19_tn14_r227_ctrl             \
Years                                                   310-340    80-110   
Variable          Description                                               
CLDTOT_ISCCP      Total cloud cover                    0.051455  0.034652   
FLDS_ISCCP        LW down SRF                          0.099944  0.041161   
FLNS_ISCCP        LW net SRF                           0.016106  0.040623   
FLUTC_CERES-EBAF  LW up Top Clearsky                  -0.073040  0.109300   
FLUT_CERES-EBAF   LW up Top                            0.072434  0.077030   
FSDS_ISCCP        SW down SRF                          0.049568  0.047349   
FSNS_ISCCP        SW net SRF                           0.036913  0.035251   
FSNTOAC_CERES     SW net TOA clearsky                 -0.090962  0.070825   
FSNTOA_CERES      SW net TOA                           0.005556  0.021231   
LHFLX_JRA25       Lat Heat Flux                       -0.020107  0.010426   
LWCF_CERES-EBAF   LW Cloud Forc                        0.014419  0.020865   
PRECT_GPCP        Precipitation                        0.035351  0.021792   
PREH2O_ERA40      Precipitable water                   0.446609  0.094264   
PSL_JRA25         SeaLev pressure                      0.135689  0.136275   
RESSURF           SRF net flux                              NaN       NaN   
RESTOA_CERES-EBAF TOA  net flux                       -0.004179 -0.011421   
RESTOM            TOmodel net flux                          NaN       NaN   
SHFLX_JRA25       Sens Heat Flux                      -0.026243  0.009865   
SST_HADISST_PI    SST preindustrial                    0.394453  0.142424   
SWCF_CERES-EBAF   SW Cloud Forc                        0.046636  0.029792   
TREFHT_JRA25      2m Temp                              0.108664  0.121712   
TS_LAND_NCEP      2m Temp land                         0.290899  0.027979   
TS_NCEP           2m Temp                              0.253790  0.067267   
U_200_JRA25       Zonal Wind JRA25                     0.204710  0.201108   
Z3_500_JRA25      Geop Height  JRA25                   0.394505  0.104396   

Run                                   N1850_f09_tn14_230218   \
Years                                                   1-20   
Variable          Description                                  
CLDTOT_ISCCP      Total cloud cover                -0.044782   
FLDS_ISCCP        LW down SRF                      -0.044087   
FLNS_ISCCP        LW net SRF                        0.071951   
FLUTC_CERES-EBAF  LW up Top Clearsky               -0.065215   
FLUT_CERES-EBAF   LW up Top                        -0.113989   
FSDS_ISCCP        SW down SRF                      -0.015239   
FSNS_ISCCP        SW net SRF                       -0.045801   
FSNTOAC_CERES     SW net TOA clearsky               0.008680   
FSNTOA_CERES      SW net TOA                       -0.104744   
LHFLX_JRA25       Lat Heat Flux                    -0.054982   
LWCF_CERES-EBAF   LW Cloud Forc                    -0.035963   
PRECT_GPCP        Precipitation                    -0.137046   
PREH2O_ERA40      Precipitable water               -0.373027   
PSL_JRA25         SeaLev pressure                  -0.114408   
RESSURF           SRF net flux                           NaN   
RESTOA_CERES-EBAF TOA  net flux                    -0.068695   
RESTOM            TOmodel net flux                       NaN   
SHFLX_JRA25       Sens Heat Flux                    0.042650   
SST_HADISST_PI    SST preindustrial                -0.345581   
SWCF_CERES-EBAF   SW Cloud Forc                    -0.122700   
TREFHT_JRA25      2m Temp                          -0.177131   
TS_LAND_NCEP      2m Temp land                     -0.226794   
TS_NCEP           2m Temp                          -0.175746   
U_200_JRA25       Zonal Wind JRA25                 -0.209790   
Z3_500_JRA25      Geop Height  JRA25               -0.367033   

Run                                   N1850_f19_tn